In [1]:
import tensorflow as tf
import librosa

2024-09-03 07:42:13.127812: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-03 07:42:13.169163: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model_path = '/shareddrive/working/model_code/models/custom_model_4/trail_1/16k_melspec-nfft-1024_a_cnn_dense_model.tflite'
file_path = '/shareddrive/working/data_code/data/neg_data/envornment/chunks/1s_chunks/env_627.wav'

In [5]:
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [7]:
sr = 16000
max_seconds = 1

In [8]:
data,_ = librosa.load(file_path,sr=sr)

In [9]:
pad_or_trunc = lambda a,i : a[0:i] if len(a) > i else a if len(a) == i else np.pad(a,(0, (i-len(a))))

def process_data(y,sr,max_seconds):
    y = pad_or_trunc(y,sr*max_seconds)
    features = librosa.feature.melspectrogram(y=y,sr=sr,n_fft=1024)
    return features

def get_features(waveform, sr):
    features = process_data(waveform,sr,max_seconds)
    shape = features.shape
    features = features.reshape([1,*shape,1])
    return features

features = get_features(data,sr=sr)

In [10]:
interpreter.set_tensor(input_details[0]['index'], features)

In [11]:
interpreter.invoke()

In [12]:
output_data = interpreter.get_tensor(output_details[0]['index'])

In [13]:
output_data

array([[0.00414084]], dtype=float32)